In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 26688044
paper_name = 'johnson_wu_2015' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/c5mt00261c1.xlsx', sheet_name='Sheet1', skiprows=2, header=None)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4874 x 7


In [7]:
original_data.head()

,0,1,2,3,4,5,6
0,BY4743 Average (n=128),0.041889,0.900905,0.041198,0.740437,0.04196,0.400476
1,YAL002W,0.039000,0.692000,0.038000,0.318000,0.03900,0.352000
2,YAL004W,0.041000,0.960000,0.037000,0.728000,0.04200,0.710000
3,YAL005C,0.040000,0.921000,0.038000,0.648000,0.03900,0.657000
4,YAL007C,0.042000,1.047000,0.037000,0.689000,0.04300,0.661000


In [8]:
original_data['orf'] = original_data[0].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                  0         1         2         3         4  \
index_input                                                                   
0            BY4743 Average (n=128)  0.041889  0.900905  0.041198  0.740437   

                   5         6                orf  
index_input                                        
0            0.04196  0.400476  BY4743AVERAGEN128  


In [12]:
original_data.set_index('orf', inplace=True)
original_data.drop(columns=[0], inplace=True)

In [13]:
# Normalize data
water = original_data[2] - original_data[1]
original_data['data2'] = (original_data[4] - original_data[3]) / water
original_data['data3'] = (original_data[6] - original_data[5]) / water

original_data = original_data.div(original_data.loc['BY4743AVERAGEN128',:])

In [14]:
original_data = original_data.replace([np.inf, -np.inf], np.nan)

In [15]:
original_data.drop(index='BY4743AVERAGEN128', inplace=True)

In [16]:
original_data = original_data[['data2','data3']].copy()

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(4841, 2)

# Prepare the final dataset

In [19]:
data = original_data.copy()

In [20]:
dataset_ids = [697, 698]
datasets = datasets.reindex(index=dataset_ids)

In [21]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [22]:
data.head()

dataset_id,697,698
data_type,value,value
orf,,
YAL002W,0.526770,1.148481
YAL004W,0.923716,1.741621
YAL005C,0.850609,1.680759
YAL007C,0.796999,1.473382
YAL008W,0.830896,1.739858


## Subset to the genes currently in SGD

In [23]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [24]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,697,698
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.526770,1.148481
1863,YAL004W,0.923716,1.741621
4,YAL005C,0.850609,1.680759
5,YAL007C,0.796999,1.473382
6,YAL008W,0.830896,1.739858


# Normalize

In [25]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [26]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [27]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,697,698,697,698
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.526770,1.148481,-0.492074,0.153596
1863,YAL004W,0.923716,1.741621,0.055430,0.865631
4,YAL005C,0.850609,1.680759,-0.045406,0.792569
5,YAL007C,0.796999,1.473382,-0.119351,0.543623
6,YAL008W,0.830896,1.739858,-0.072596,0.863514


# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 26688044...
Inserting the new data...


100%|██████████| 2/2 [00:15<00:00,  7.55s/it]

Updating the data_modified_on field...
